In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [5]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [82]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [100]:
from sklearn.model_selection import GridSearchCV
param_grid={'n_estimators':[10,100,200,500],"learning_rate":[0.01,0.1,0.5,1.0]}
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 1.0],
                         'n_estimators': [10, 100, 200, 500]},
             scoring='f1_weighted', verbose=3)

In [102]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [104]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'learning_rate': 1.0, 'n_estimators': 10}: 0.7553813257305774


In [106]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[ 7  5]
 [11 38]]


In [108]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.39      0.58      0.47        12
           1       0.88      0.78      0.83        49

    accuracy                           0.74        61
   macro avg       0.64      0.68      0.65        61
weighted avg       0.79      0.74      0.76        61



In [110]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8078231292517006

In [112]:
table=pd.DataFrame.from_dict(re)

In [114]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015473,0.002041,0.005408,0.001498,0.01,10,"{'learning_rate': 0.01, 'n_estimators': 10}",0.833016,0.728557,0.838542,0.719956,0.871978,0.798410,0.062056,6
1,0.139509,0.006239,0.016118,0.000588,0.01,100,"{'learning_rate': 0.01, 'n_estimators': 100}",0.767366,0.728557,0.750000,0.719956,0.587398,0.710655,0.063817,16
2,0.283441,0.009509,0.029937,0.000490,0.01,200,"{'learning_rate': 0.01, 'n_estimators': 200}",0.767366,0.765833,0.750000,0.750000,0.747151,0.756070,0.008674,13
3,0.639621,0.013761,0.068534,0.002130,0.01,500,"{'learning_rate': 0.01, 'n_estimators': 500}",0.767366,0.840000,0.824074,0.701282,0.808772,0.788299,0.049764,9
4,0.013415,0.000198,0.004907,0.001987,0.10,10,"{'learning_rate': 0.1, 'n_estimators': 10}",0.833016,0.728557,0.750000,0.719956,0.808772,0.768060,0.044894,10
5,0.130982,0.003472,0.016020,0.000314,0.10,100,"{'learning_rate': 0.1, 'n_estimators': 100}",0.767366,0.916508,0.877273,0.701282,0.808772,0.814240,0.076705,2
6,0.246398,0.007432,0.028834,0.001473,0.10,200,"{'learning_rate': 0.1, 'n_estimators': 200}",0.767366,0.916508,0.877273,0.701282,0.808772,0.814240,0.076705,2
7,0.600685,0.008803,0.066574,0.001951,0.10,500,"{'learning_rate': 0.1, 'n_estimators': 500}",0.767366,0.840000,0.916667,0.750000,0.771704,0.809147,0.061901,5
8,0.013516,0.001766,0.003405,0.000378,0.50,10,"{'learning_rate': 0.5, 'n_estimators': 10}",0.767366,0.840000,0.877273,0.750000,0.824074,0.811743,0.046960,4
9,0.119606,0.002176,0.015122,0.000493,0.50,100,"{'learning_rate': 0.5, 'n_estimators': 100}",0.767366,0.840000,0.916667,0.750000,0.701282,0.795063,0.075365,7
